In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_core.output_parsers.string import StrOutputParser

from langchain.schema.runnable import RunnablePassthrough
from langchain_groq import ChatGroq

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
#llm = ChatOpenAI(temperature=0, openai_api_key = openai_api_key)
llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0)

In [8]:
original_review_prompt = ChatPromptTemplate.from_template("""
Translate the following review to English. If the review is already in English,
then output the review as is : {review}
""")

In [9]:
summary_creation_prompt = ChatPromptTemplate.from_template("""
Create a concise summary of the following in few sentences:
{english_review}
""")

In [10]:
original_language_prompt = ChatPromptTemplate.from_template("""
What language is the fllowing review in? : \n\n {review}
""")

In [13]:
find_sentiment_prompt = ChatPromptTemplate.from_template("""
You are an expert in reading text and identifying the sentiment of the customer
who wrote the text. Using all of the inputs below, find the stentiment of the review.

Review : {review}
English Review : {english_review}
Summary : {summary}
Original Review Language : {review_language}

Determine the sentiment of the review.
""")

In [15]:
build_email_response_prompt = ChatPromptTemplate.from_template("""
You are an expert customer service agent who handles a lot of angry, unhappy as well
happy and polite customers. Your task is to generate an email response for a customer who
has written a review. Your response should thank them for their feedback.
If the sentiment of the review is positive or neutral, thank them for being an esteemed customer.
If the sentiment is negative, apologize for the inconvinience and suggest them to
reach out to the customer care team for resolution. In your response be very specific and
informative by using the specific details from the customer feedback.
Your response should be concise and professional.
Sign the email with your name as "Generative Geek".

Review : {review}
English Review : {english_review}
Summary : {summary}
Original Review Language : {review_language}
Sentiment : {sentiment}

""")

In [16]:
original_review_chain = original_review_prompt | llm | StrOutputParser()
summary_creation_chain = summary_creation_prompt | llm | StrOutputParser()
original_language_chain = original_language_prompt | llm | StrOutputParser()
find_sentiment_chain = find_sentiment_prompt | llm | StrOutputParser()
build_email_response_chain = build_email_response_prompt | llm | StrOutputParser()

In [17]:
final_chain = ({"english_review":original_review_chain, "review": RunnablePassthrough()}
               | RunnablePassthrough.assign(summary = summary_creation_chain)
               | RunnablePassthrough.assign(review_language = original_language_chain)
               | RunnablePassthrough.assign(sentiment = find_sentiment_chain)
               | RunnablePassthrough.assign(automated_email = build_email_response_chain)
               )

In [23]:
review = """
The fabric is really cheap and uncomfortable. After just one wash, it shrank and the color faded. The stitching is already coming apart, and the fit is nothing like what was advertised. Definitely not worth the price.

"""

In [24]:
result = final_chain.invoke(review)

In [25]:
result

{'english_review': 'The review is already in English, so the output is:\n\nThe fabric is really cheap and uncomfortable. After just one wash, it shrank and the color faded. The stitching is already coming apart, and the fit is nothing like what was advertised. Definitely not worth the price.',
 'review': '\nThe fabric is really cheap and uncomfortable. After just one wash, it shrank and the color faded. The stitching is already coming apart, and the fit is nothing like what was advertised. Definitely not worth the price.\n\n',
 'summary': 'The product has poor quality fabric that is uncomfortable and prone to shrinkage and fading. The stitching is also weak, and the fit does not match the advertised description, making it overpriced.',
 'review_language': 'The language of the review is English.',
 'sentiment': 'The sentiment of the review is extremely negative. The customer expresses strong dissatisfaction with the product, citing several issues with its quality, including:\n\n1. Cheap

In [27]:
def print_review_response_email(review):
  print("Analysing....")
  result = final_chain.invoke(review)
  print("Analysis done.")
  print("\n")

  print(f"Review: {review}")
  print("\n")
  print(f"Sentiment: {result['sentiment']}")
  print("\n")
  print(f"Automated Email: \n\n{result['automated_email']}")
  print("\n")

In [28]:
print_review_response_email(review)

Analysing....
Analysis done.


Review: 
The fabric is really cheap and uncomfortable. After just one wash, it shrank and the color faded. The stitching is already coming apart, and the fit is nothing like what was advertised. Definitely not worth the price.




Sentiment: The sentiment of the review is extremely negative. The customer expresses strong dissatisfaction with the product, citing several issues with the fabric, stitching, and fit. They use words like "cheap", "uncomfortable", "shrank", "faded", and "coming apart" to describe the product's poor quality. Additionally, they state that the product is "not worth the price", indicating that they feel it is overpriced for its quality. Overall, the tone of the review is critical and disapproving, indicating a strongly negative sentiment.


Automated Email: 

Subject: Concern with Recent Purchase - Apology and Resolution

Dear [Customer's Name],

Thank you for taking the time to share your feedback about your recent purchase. I apol

In [ ]:
review = """
Ich bin absolut begeistert von meiner neuen Waschmaschine! Sie übertrifft meine Erwartungen in jeder Hinsicht. Die innovative Technologie sorgt dafür, dass meine Wäsche immer perfekt sauber und frisch herauskommt. Besonders beeindruckt hat mich die Vielzahl an Programmen, die für jede Art von Textilien geeignet sind.

Die Bedienung ist kinderleicht, und die Maschine arbeitet angenehm leise. Der Energiesparmodus und die hohe Effizienz haben meine Stromrechnung bereits positiv beeinflusst. Der Kundenservice war ebenfalls erstklassig – schnell, hilfsbereit und professionell.

Vielen Dank an [Unternehmen], dass Sie ein so herausragendes Produkt entwickelt haben. Ich werde es definitiv all meinen Freunden und meiner Familie weiterempfehlen!

"""

In [ ]:
print_review_response_email(review)

Analysing....
Analysis done.


Review: 
Ich bin absolut begeistert von meiner neuen Waschmaschine! Sie übertrifft meine Erwartungen in jeder Hinsicht. Die innovative Technologie sorgt dafür, dass meine Wäsche immer perfekt sauber und frisch herauskommt. Besonders beeindruckt hat mich die Vielzahl an Programmen, die für jede Art von Textilien geeignet sind.

Die Bedienung ist kinderleicht, und die Maschine arbeitet angenehm leise. Der Energiesparmodus und die hohe Effizienz haben meine Stromrechnung bereits positiv beeinflusst. Der Kundenservice war ebenfalls erstklassig – schnell, hilfsbereit und professionell.

Vielen Dank an [Unternehmen], dass Sie ein so herausragendes Produkt entwickelt haben. Ich werde es definitiv all meinen Freunden und meiner Familie weiterempfehlen!




Sentiment: The sentiment of the review is overwhelmingly positive. The reviewer is thrilled with their new washing machine, praising its performance, features, ease of use, and energy efficiency. They also expr

In [ ]:
result = final_chain.invoke(review)
result

{'english_review': 'I am absolutely thrilled with my new washing machine! It exceeds my expectations in every way. The innovative technology ensures that my laundry always comes out perfectly clean and fresh. I was particularly impressed by the variety of programs that are suitable for all types of fabrics.\n\nThe operation is very easy, and the machine works pleasantly quietly. The energy-saving mode and high efficiency have already had a positive impact on my electricity bill. The customer service was also excellent - fast, helpful, and professional.\n\nThank you to [Company] for developing such an outstanding product. I will definitely recommend it to all my friends and family!',
 'review': '\nIch bin absolut begeistert von meiner neuen Waschmaschine! Sie übertrifft meine Erwartungen in jeder Hinsicht. Die innovative Technologie sorgt dafür, dass meine Wäsche immer perfekt sauber und frisch herauskommt. Besonders beeindruckt hat mich die Vielzahl an Programmen, die für jede Art von 